<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Flux-variablity-analysis-(Part-1)" data-toc-modified-id="Flux-variablity-analysis-(Part-1)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Flux variablity analysis (Part 1)</a></span><ul class="toc-item"><li><span><a href="#How-does-FVA-work?" data-toc-modified-id="How-does-FVA-work?-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>How does FVA work?</a></span></li><li><span><a href="#What-does-flux-variablity-analysis-do?" data-toc-modified-id="What-does-flux-variablity-analysis-do?-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>What does flux variablity analysis do?</a></span></li><li><span><a href="#Analyzing-the-E.-coli-core-model" data-toc-modified-id="Analyzing-the-E.-coli-core-model-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Analyzing the <em>E. coli</em> core model</a></span><ul class="toc-item"><li><span><a href="#Loopless-version-of-FVA" data-toc-modified-id="Loopless-version-of-FVA-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Loopless version of FVA</a></span></li></ul></li><li><span><a href="#Run-flux-variability-analysis-for-optimally-growing-E.-coli" data-toc-modified-id="Run-flux-variability-analysis-for-optimally-growing-E.-coli-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Run flux variability analysis for optimally growing <em>E. coli</em></a></span></li><li><span><a href="#Exercises-(20-min)" data-toc-modified-id="Exercises-(20-min)-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Exercises (20 min)</a></span><ul class="toc-item"><li><span><a href="#Exercise-1" data-toc-modified-id="Exercise-1-1.5.1"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Exercise 1</a></span></li><li><span><a href="#Exercise-2" data-toc-modified-id="Exercise-2-1.5.2"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span>Exercise 2</a></span></li></ul></li></ul></li><li><span><a href="#Phenotypic-phase-plane-analysis-(Part-2)" data-toc-modified-id="Phenotypic-phase-plane-analysis-(Part-2)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Phenotypic phase plane analysis (Part 2)</a></span><ul class="toc-item"><li><span><a href="#Exercise-(10-min)" data-toc-modified-id="Exercise-(10-min)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Exercise (10 min)</a></span></li></ul></li><li><span><a href="#References" data-toc-modified-id="References-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>References</a></span></li></ul></div>

# Flux variablity analysis (Part 1)

A number of cell factory design algorithms use flux variablity analysis (FVA) as a pre-processing step, so it is worth taking some time to understand the problem it is designed to solve, how it works, and what you can do with it.

## How does FVA work?


Conceptually, FVA works by looping through each reaction in the network, and solving for the maximum flux and then solving for the minimum flux associated with that reaction. 

$$\begin{array}{lll}
\text{for $i$ in $1..n$} & \\
&    \underset{\vec{v}}{\mbox{minimize}}   & v_{i} \\
&    \mbox{subject to}  & S\cdot \vec{v} = 0 \\
&    & 0 \leq \vec{v}  \\
&    & v_1 \leq 10 \\
&    \underset{\vec{v}}{\mbox{maximize}}   & v_{i} \\
&    \mbox{subject to}  & S\cdot \vec{v} = 0 \\
&    & 0 \leq \vec{v}  \\
&    & v_1 \leq 10 \\
\end{array}$$


In [ ]:
import pandas as pd

def fva( model ):
    fva = {'minimum':{},
           'maximum':{}}
    for reaction in model.reactions:
        with model:
            model.objective = {reaction: -1}
            fva['minimum'][reaction.id] = model.slim_optimize()
        with model:
            model.objective = {reaction: 1}
            fva['maximum'][reaction.id] =  model.slim_optimize()
    return pd.DataFrame(fva)[['minimum','maximum']]

## What does flux variablity analysis do?
Although each reaction has explicit lower and upper bounds, sometimes the constraint on one reaction imposes implicit constraints on other reactions.  Flux variability analysis is an estimate of these implicit bounds.
Why do I say an estimate?  Because the implicit bounds may actually be tighter than what flux variability analysis predicts.  This can be seen in the figure from [[Mahadevan 2003](#references)] 
![FVA](FVA.gif)

As you can see, when there are only two fluxes, FVA forms the tightest rectangle around the actual solution, which is a polygon. FVA forms a parallelopiped around the actual solution, which is a polyhedra in 3 dimensions, and in general, FVA forms a hyperrectangle around the polytope in $n$ dimensions. The work required to find an exact solution, unfortunately, grows exponentially in the number of reactions, but for genome-scale models, FVA is usually good enough.

In [ ]:
import escher

abc_model =  load_json_model('ABC/ABC_model.json') 
abc_model.reactions.R_1.upper_bound=10
abc_fva = fva(abc_model)
display(abc_fva)
escher.Builder(map_json='ABC/abc_map.json',
               model=abc_model,
               reaction_data=abc_fva['maximum'].to_dict(),
              ).display_in_notebook()

    

As you can see, explicitly constraining the uptake rate on $A$ induces implicit constraints on all the other fluxes in the ABC network.  

## Analyzing the *E. coli* core model
For the rest of this episode, we will be using the *E. coli* core model, which is not as enormous as a full genome-scale model, but is still complex enough to produce nontrival results when analyzed.


In [ ]:
import escher
from cobra.io import read_sbml_model, load_json_model
from cobra.flux_analysis import flux_variability_analysis
model = read_sbml_model('data/e_coli_core.xml.gz')

In [ ]:
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

The algorithm described is $O(2n)$ where $n$ is the number of reactions, but the actual implementation in Cobra is actually much faster [[Gudmundsson 2010](#references)].

In [ ]:
%%time
result = flux_variability_analysis(model)

Inspect the result.

In [ ]:
%time result = fva(model)

Get an overview of a few key statistics of the resulting flux ranges.

In [ ]:
result.describe()

Note that the max maximum flux is 1000.  This doesn't seem realistic. Let's visualize the flux ranges on a pathway map of _E. coli's_ central carbon metabolism.

In [ ]:
abs_flux_ranges = abs(result.maximum - result.minimum).to_dict()
escher.Builder('e_coli_core.Core metabolism', reaction_data=abs_flux_ranges).display_in_notebook()

Those reactions showing up in red are futile cyles.

In [ ]:
result[result.maximum > 500]

### Loopless version of FVA

In [ ]:
result_no_cycles = flux_variability_analysis(model, loopless=True)
result_no_cycles.describe()

In [ ]:
abs_flux_ranges = abs(result_no_cycles.maximum - result_no_cycles.minimum).to_dict()
escher.Builder('e_coli_core.Core metabolism', reaction_data=abs_flux_ranges).display_in_notebook()

## Run flux variability analysis for optimally growing _E. coli_

(Optimal) Flux Balance Analysis solutions are not necessariliy unique. Flux Variablity Analysis is a good tool for estimating the space of alternative optimal solutions.

In [ ]:
fba_solution = model.optimize()

In [ ]:
fba_solution.objective_value

In [ ]:
model_optimal = model.copy()

In [ ]:
model_optimal.reactions.BIOMASS_Ecoli_core_w_GAM.lower_bound = fba_solution.objective_value

In [ ]:
result_max_obj = flux_variability_analysis(model_optimal, loopless=True)

In [ ]:
result_max_obj

This is actually such a common task that `flux_variability_analysis` provides an option for fixing the objective's flux at a certain percentage.

In [ ]:
result_max_obj = flux_variability_analysis(model, fraction_of_optimum=1., loopless=True)

In [ ]:
result_max_obj

Turns out that in this small core metabolic model, the optimal solution is actually unique!

In [ ]:
sum(abs(result_max_obj.minimum - result_max_obj.maximum))

## Exercises (20 min)

### Exercise 1

Explore how relaxing the constraint on the growth rate affects the solution space:
1. Modify the code to explore flux ranges for $\mu \gt 0.7 \ h^{-1}$ 
1. Plot the sum of flux ranges over a range of percentages.

### Exercise 2

Using FVA, determine all blocked reactions ($v = 0$) in the model.

# Phenotypic phase plane analysis (Part 2)

Load a few packages. Cobrapy implements phenotypic phase plane calculation as well but the cameo version comes with better plotting capabilities.

In [ ]:
import pandas
pandas.options.display.max_rows = 12
from cameo.flux_analysis import phenotypic_phase_plane

Compute the phenotypic phase plane for growth and acetate secretion.

In [ ]:
result = phenotypic_phase_plane(model,
                                variables=[model.reactions.BIOMASS_Ecoli_core_w_GAM],
                                objective=model.reactions.EX_ac_e)

Look at the result in a tabular view.

In [ ]:
result.data_frame

Plot the phenotypic phase plane showing the flux through the objective.

In [ ]:
result.plot()

We may also be interested in other parameters such as the carbon yield for output versus input at different growth rates.

In [ ]:
result.plot(estimate='c_yield')

We can also calculate a three dimensional phenotypic phase plane to compare the influence of aerobic and anaerobic environements.

In [ ]:
result_3D = phenotypic_phase_plane(model, variables=[model.reactions.EX_ac_e, model.reactions.EX_o2_e],
                                   objective=model.reactions.BIOMASS_Ecoli_core_w_GAM, points=50)

In [ ]:
result_3D.data_frame

In [ ]:
result_3D.plot()

## Exercise (10 min)

* Use `phenotypic_phase_plane` to determine the optimal O2 uptake rate.

<a id="references"></a>
# References

* [Mahadevan 2003]: Mahadevan R, Schilling C: [The effects of alternate optimal solutions in constraint-based genome-scale metabolic models](http://www.ncbi.nlm.nih.gov/pubmed/14642354). Metabolic engineering 2003, 5(4):264–276. 10.1016/j.ymben.2003.09.002
* [Gudmundsson 2010]: Gudmundsson, S., Thiele, I. Computationally efficient flux variability analysis. BMC Bioinformatics. 11, 489 (2010).